## Importing the goodies 

In [1]:
import torch
import torchvision
import torch.nn as nn                                              
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(linewidth = 120)
torch.set_grad_enabled(True)
import torchvision
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import pyautogui
from PIL import Image 
from Xlib import display
import numpy as np
import matplotlib.pyplot as plt
import time
from mss.linux import MSS as mss
import mss.tools
import cv2
import torchvision.transforms as transforms

## Defining the nn layers and loading up the trained model

In [42]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding = 1)
        self.pool = nn.MaxPool2d(3, 3)
        
        self.dropout = nn.Dropout(p=0.5)
        self.conv2 = nn.Conv2d(16, 32 , 3, padding = 1)
        # self.conv3 = nn.Conv2d(64, 64 , 3, padding = 1)

        
        self.fc1 = nn.Linear(512,256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)

        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = (self.pool(F.relu(self.conv1(x))))
        x = self.pool(F.relu(self.conv2(x)))
        # x = self.pool(F.relu(self.conv3(x)))


        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x =F.relu(self.fc3(x))

        x = self.fc4(x)

        return x

drunk_driver = Net()
print(drunk_driver)

drunk_driver.load_state_dict(torch.load("torch/drunk_driver.pt", map_location=torch.device('cpu')))

Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
)


<All keys matched successfully>

## Defining some utilities 

In [39]:

def find_mouse(x = False , y = False):
    
    data = display.Display().screen().root.query_pointer()._data
    res = [data["root_x"], data["root_y"]]
    if x == True and y == True:
        return res
    elif x == True and y == False:
        return res[0]
    else:
        return res[1]

def move_mouse_co_ord_x(x_val , time = 0.001):
    
    current_y = find_mouse(y = True)
    pyautogui.moveTo(x_val, current_y, duration= time )
    

def screen_record_efficient(top = 225, left = 0, width = 800, height = 260, resize_width = 80, resize_height = 25):
  
    mon = {"top": top, "left": left, "width": width, "height": height}
    sct = mss.mss()
    img = np.array(sct.grab(mon))
    img = cv2.resize(img, (80,25))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    sct.close()
    return img

loader = transforms.Compose([transforms.ToTensor()])

def numpy_to_tensor(np_arr):
    image = loader(np_arr).float().unsqueeze(0)
    return image

def flex(np_img, turn, window_name = "foo"):

    np_img = cv2.cvtColor(np_img, cv2.COLOR_BGR2RGB)
    np_img = cv2.resize(np_img, (500, 150))
    x_line = int(250 - (turn))
    np_img = cv2.line(np_img, (250, 149),(x_line, 0), color = (0, 255,0), thickness= 10) 
    cv2.imshow(window_name, np_img) 
    

def canny(gray):
    im = cv2.Canny(gray,50,150,apertureSize = 3)
    return im
    
def segment_road_on_np(image, lower_gray = [0, 0, 1], higher_gray = [179, 50, 200]):  ## obsolete, delete this 
    img = (image).astype(np.uint8)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_gray = np.array(lower_gray, np.uint8)
    upper_gray = np.array(higher_gray, np.uint8)
    mask_gray = cv2.inRange(hsv, lower_gray, upper_gray)
    img_res = cv2.bitwise_and(img, img, mask = mask_gray)
    return img_res



## And it all comes together here
> * Feel free to change the vertices in `screen_record_efficient()` depending on where you place the game window on the screen
*  Drag mouse to the corner of the screen to quit

In [58]:

## top = 370,width = 800, height = 110, resize_width = 80, resize_height = 25

def not_so_drunk_driver(flex_mode = False, segment_road = True):
    
    print ("running...")
    print ("drag mouse to the corner of the screen to terminate -")
    while True:
        try:
            foo = screen_record_efficient(top = 225,width = 800, height = 142, resize_width = 80, resize_height = 25)  ## FPV TRACK VIEW  

            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

            if segment_road == True:
                foo =  cv2.blur(canny(foo), (3,3))



            foo_tensor = numpy_to_tensor(foo)
            steer = int(drunk_driver(foo_tensor).item())

            if flex_mode == True:

                flex( foo, turn = steer, window_name = "segmented")
    #                 flex( foo, turn = steer, window_name = "raw")

            x_co = 400 - steer*1.2

            move_mouse_co_ord_x(x_co , time = 0)
        except: 

            print("end ----")
            break

In [59]:
not_so_drunk_driver(flex_mode = True, segment_road = True)  


running...
drag mouse to the corner of the screen to terminate -
end ----


In [24]:
cv2.destroyAllWindows()
